<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem_13_3D_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# ============================================================
# problem13.m  --- 3D Frame FEM (Python / Colab)
# ============================================================

# -------------------------
# Material & section
# -------------------------
E  = 210e6
A  = 0.02
Iy = 10e-5
Iz = 20e-5
J  = 5e-5
G  = 84e6

# -------------------------
# Node coordinates
# -------------------------
nodeCoordinates = np.array([
    [0, 0, 0],   # 1
    [0, 0, 4],   # 2
    [4, 0, 4],   # 3
    [4, 0, 0],   # 4
    [0, 5, 0],   # 5
    [0, 5, 4],   # 6
    [4, 5, 4],   # 7
    [4, 5, 0],   # 8
], dtype=float)

# -------------------------
# Element connectivity
# -------------------------
elementNodes = np.array([
    [1, 5],
    [2, 6],
    [3, 7],
    [4, 8],
    [5, 6],
    [6, 7],
    [7, 8],
    [8, 5],
], dtype=int)

numberNodes    = nodeCoordinates.shape[0]
numberElements = elementNodes.shape[0]
GDoF = 6 * numberNodes

# -------------------------
# Global vectors
# -------------------------
U = np.zeros((GDoF, 1))
force = np.zeros((GDoF, 1))
stiffness = np.zeros((GDoF, GDoF))

# Force vector (MATLAB: force(37) = -15)
force[36, 0] = -15.0

# Boundary conditions (MATLAB: prescribedDof = 1:24)
prescribedDof = np.arange(0, 24)
freeDof = np.setdiff1d(np.arange(GDoF), prescribedDof)

# ============================================================
# FEM FUNCTIONS
# ============================================================

def element_dofs(n1, n2):
    i1 = n1 - 1
    i2 = n2 - 1
    return np.hstack([
        np.arange(6*i1, 6*i1+6),
        np.arange(6*i2, 6*i2+6)
    ])

def lambda_matrix(x1,y1,z1,x2,y2,z2):
    L = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)

    if np.isclose(x1,x2) and np.isclose(y1,y2):
        if z2 > z1:
            return np.array([[0,0,1],[0,1,0],[-1,0,0]]), L
        else:
            return np.array([[0,0,-1],[0,1,0],[1,0,0]]), L

    Cxx = (x2-x1)/L
    CYx = (y2-y1)/L
    CZx = (z2-z1)/L
    D = np.sqrt(Cxx**2 + CYx**2)

    Lambda = np.array([
        [ Cxx,      CYx,       CZx],
        [-CYx/D,    Cxx/D,     0  ],
        [-Cxx*CZx/D,-CYx*CZx/D, D ]
    ])
    return Lambda, L

def local_stiffness(E,A,Iz,Iy,G,J,L):
    k1  = E*A/L
    k2  = 12*E*Iz/L**3
    k3  = 6*E*Iz/L**2
    k4  = 4*E*Iz/L
    k5  = 2*E*Iz/L
    k6  = 12*E*Iy/L**3
    k7  = 6*E*Iy/L**2
    k8  = 4*E*Iy/L
    k9  = 2*E*Iy/L
    k10 = G*J/L

    a = np.diag([k1, k2, k6])
    b = np.array([[0,0,0],[0,0,k3],[0,-k7,0]])
    c = np.diag([k10,k8,k4])
    d = np.diag([-k10,k9,k5])

    return np.block([
        [ a,  b, -a,  b ],
        [ b.T,c,  b,  d ],
        [-a, b.T, a, -b ],
        [ b, d.T,-b.T,c ]
    ])

def formStiffness3Dframe():
    K = np.zeros((GDoF, GDoF))
    for e in range(numberElements):
        n1, n2 = elementNodes[e]
        x1,y1,z1 = nodeCoordinates[n1-1]
        x2,y2,z2 = nodeCoordinates[n2-1]

        Lambda, L = lambda_matrix(x1,y1,z1,x2,y2,z2)
        k_local = local_stiffness(E,A,Iz,Iy,G,J,L)

        Z = np.zeros((3,3))
        R = np.block([
            [Lambda,Z,Z,Z],
            [Z,Lambda,Z,Z],
            [Z,Z,Lambda,Z],
            [Z,Z,Z,Lambda]
        ])

        k_global = R.T @ k_local @ R
        edof = element_dofs(n1,n2)
        K[np.ix_(edof,edof)] += k_global
    return K

# ============================================================
# SOLUTION
# ============================================================

stiffness = formStiffness3Dframe()

Kff = stiffness[np.ix_(freeDof, freeDof)]
Ff  = force[freeDof]

Uf = np.linalg.solve(Kff, Ff)
U[freeDof] = Uf

# ============================================================
# OUTPUT (giống MATLAB)
# ============================================================
print("=== DISPLACEMENTS ===")
for i in range(GDoF):
    print(f"DOF {i+1:2d}: {U[i,0]: .8e}")


=== DISPLACEMENTS ===
DOF  1:  0.00000000e+00
DOF  2:  0.00000000e+00
DOF  3:  0.00000000e+00
DOF  4:  0.00000000e+00
DOF  5:  0.00000000e+00
DOF  6:  0.00000000e+00
DOF  7:  0.00000000e+00
DOF  8:  0.00000000e+00
DOF  9:  0.00000000e+00
DOF 10:  0.00000000e+00
DOF 11:  0.00000000e+00
DOF 12:  0.00000000e+00
DOF 13:  0.00000000e+00
DOF 14:  0.00000000e+00
DOF 15:  0.00000000e+00
DOF 16:  0.00000000e+00
DOF 17:  0.00000000e+00
DOF 18:  0.00000000e+00
DOF 19:  0.00000000e+00
DOF 20:  0.00000000e+00
DOF 21:  0.00000000e+00
DOF 22:  0.00000000e+00
DOF 23:  0.00000000e+00
DOF 24:  0.00000000e+00
DOF 25:  1.47458723e-04
DOF 26:  3.29332096e-06
DOF 27:  1.25446812e-03
DOF 28:  8.09687160e-05
DOF 29: -5.05273916e-05
DOF 30: -1.45783376e-05
DOF 31: -1.19183524e-03
DOF 32: -6.70878569e-06
DOF 33:  1.24289133e-03
DOF 34:  7.88306841e-05
DOF 35: -5.05797386e-05
DOF 36:  1.24319734e-04
DOF 37: -1.19910351e-03
DOF 38:  7.51875769e-07
DOF 39:  1.12019091e-03
DOF 40:  7.09130651e-05
DOF 41: -1.1639134